In [22]:
from datetime import datetime
from app import crud, models, schemas
from app.db.session import SessionLocal
from sqlalchemy.orm import Session
from sqlalchemy.sql import func
from app.services.controlling import ControllingCalendar

db = SessionLocal()
calendar = ControllingCalendar(year=2020, monthrange=[7,12]).get()
start = calendar['range']['start']
end = calendar['range']['end']

def get_clean_discharge(
        _db: Session = db, *, start: datetime = None, end: datetime = None,
        store_internal_id: int = None, owner_id: int = None
) -> float:
    """
    TO AVOID MULTIPLE COUNTED DISCHARGES
    WE ONLY ACCEPT ONE DISCHARGE PER CONTRACT
    MEANS WE VE TO EXCLUDE ALL OTHER ROWS WITH THE SAME CONTRACT NR
    """
    exclude_contracts: t.List[int] = []
    discharges: t.List[float] = []
    #  FOR BETTER SQL PERFORMANCE WE ENCLOSE OUR QUERY: EXCLUDE ALL WITH DISCHARGE IS NULL;
    claims = crud.claim.get_query(_db)
    if start and end:
        claims = claims.filter(models.Claim.created_at.between(start, end))  # noqa
    if store_internal_id:
        claims = claims.filter(models.Claim.store_internal_id == store_internal_id)  # noqa
    if owner_id:
        claims = claims.filter(models.Claim.owner_id == owner_id)  # noqa
    claims = claims.filter(models.Claim.discharge.isnot(None))  # noqa
    claims: t.List[models.Claim] = claims.all()  # noqa

    for claim in claims:
        if claim.contract_nr not in exclude_contracts:
            exclude_contracts.append(claim.contract_nr)
            discharges.append(claim.discharge)
    total_discharges: float = sum(discharges)
    total_discharges_round: float = round(total_discharges, 2)
    return total_discharges_round

In [36]:
def get_average_claim_bill(
        _db: Session = db, *, start: datetime = None, end: datetime = None,
        store_internal_id: int = None, owner_id: int = None
):
    query = crud.claim.get_query(_db)
    if start and end:
        query = query.filter(models.Claim.created_at.between(start, end))
    if store_internal_id:
        query = query.filter(models.Claim.store_internal_id == store_internal_id)
    if owner_id:
        query = query.filter(models.Claim.owner_id == owner_id)
    bill = query.with_entities(func.avg(models.Claim.bill).label("_bill")).first()
    bill_round = round(bill[0], 2)
    
    return bill_round

In [45]:
x = get_average_claim_bill(start=start, end=end, store_internal_id=188)

In [46]:
y = get_clean_discharge(start=start, end=end, store_internal_id=188)

-5956.37